In [ ]:
 pip install -r requirements.txt

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2854264/")
page_data = loader.load().pop().page_content
print(page_data)

Senior Software Engineer, Emerging Device Software - Job ID: 2854264 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsDiversity, equity, and inclusionInterview tipsLeadership principlesWorking at AmazonFAQ×Senior Software Engineer, Emerging Device SoftwareJob ID: 2854264 | ADCI - KarnatakaApply nowDESCRIPTIONAmazon lab126 Platform Software team is looking for Senior Software Development Engineer to lead the design and architecture of new consumer devices. You will drive technical direction and feature roadmap for next gen devices.Key job responsibilities- Develop high quality embedded software for Consumer products.- Lead key components in the area of Middleware and frameworks layers- Deal with ambiguity, make data-driven decisions and juggle cross-organization complexity- Evaluate different design choices, prototype new technology, and implement technical soluti

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})


from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Software Engineer, Emerging Device Software',
 'experience': '5+ years of non-internship professional software development experience',
 'skills': ['C',
  'C++',
  'Linux',
  'RTOS',
  'Middleware',
  'Android',
  'Data structures',
  'Algorithms',
  'Concurrency',
  'Code optimization'],
 'description': 'Develop high quality embedded software for Consumer products, lead key components in the area of Middleware and frameworks layers, and contribute in defining product roadmap.'}

In [ ]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
job = json_res
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])


In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Virat, a business development executive at York. York is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase York's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Embedded Software Development Services for Consumer Products

Dear Hiring Manager,

I came across the job description for a Senior Software Engineer, Emerging Device Software, and I believe AtliQ can fulfill your requirements. With 5+ years of experience in non-internship professional software development, our team of experts possesses the necessary skills to develop high-quality embedded software for consumer products.

Our team is well-versed in C, C++, Linux, RTOS, Middleware, Android, Data structures, Algorithms, Concurrency, and Code optimization, making us an ideal fit for leading key components in the area of Middleware and frameworks layers. We can contribute to defining your product roadmap and ensure seamless integration of business processes through automated tools.

At AtliQ, we have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our portfolio includes:
